In [2]:
import pandas as pd
import ast
import numpy as np
from tqdm import tqdm
from sklearn.cluster import KMeans

In [22]:
df_inversores = pd.read_excel('data_investors.xlsx')

In [4]:
base_emprendedores1 = pd.read_csv('Grid view.csv')
base_emprendedores2 = pd.read_csv('Grid view (1).csv')
interseccion = [col for col in base_emprendedores2.columns.tolist() if col in base_emprendedores1.columns.tolist()]
df_emprendedores = pd.concat([base_emprendedores1[interseccion], base_emprendedores2[interseccion]], ignore_index=True)

In [9]:
def clusterizar(data):
    def str_to_list(string):
        try:
            lista_final = ast.literal_eval(string)
        except:
            lista_final = np.nan
        return lista_final
    data['Fund Type'] = data['Fund Type'].apply(str_to_list)
    data['Fund Stage'] = data['Fund Stage'].apply(str_to_list)
    data['Fund Focus (Sectors)'] = data['Fund Focus (Sectors)'].apply(str_to_list)
    data['Location'] = data['Location'].apply(str_to_list)
    data['Portfolio Companies'] = data['Portfolio Companies'].apply(lambda x: x.split(', ') if type(x) == str else x)
    
    fund_types = list(set([j for i in data["Fund Type"] if type(i) != float for j in i]))
    fund_stages = list(set([j for i in data["Fund Stage"] if type(i) != float for j in i]))
    fund_sectors = list(set([j for i in data["Fund Focus (Sectors)"] if type(i) != float for j in i]))
    locations = list(set([j for i in data["Location"] if type(i) != float for j in i]))
    companies = list(set([j for i in data["Portfolio Companies"] if type(i) != float for j in i]))

    df_train = pd.DataFrame( columns= ['name']+fund_sectors)

    for index, row in tqdm(data.iterrows(), total= len(data)):
        new_row = {k:v for k,v in zip(df_train.columns, [[0]]*len(df_train.columns))}
        new_row['name'] = [row['Investor Name']]

        if type(row['Fund Focus (Sectors)']) == list:
            for fund in  row['Fund Focus (Sectors)']:
                new_row[fund] = [1]

        df_train = pd.concat([df_train, pd.DataFrame(new_row)])


    df_predict = df_train.copy()
    kmeans = KMeans(random_state=0, n_init="auto").fit(df_train.drop('name', axis = 1))
    kmeans.labels_
    df_predict['cluster'] = kmeans.predict(df_train.drop('name', axis = 1))

    return df_predict['cluster']

In [10]:
clusterizar(data_original)

100%|██████████| 2268/2268 [00:12<00:00, 183.63it/s]


0    6
0    5
0    1
0    2
0    6
    ..
0    6
0    6
0    6
0    6
0    6
Name: cluster, Length: 2268, dtype: int32